In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Step 1: Load the CSV Dataset
csv_path = r"E:\Data set\vehicle dataset\train\annotations.csv"
images_dir = r"E:\Data set\vehicle dataset\train\images"

data = pd.read_csv(csv_path)

# Step 2: Preprocess the Data
# Convert labels to integers
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

# Split the dataset into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])

# Function to find the correct image file with multiple extensions
def find_image_file(img_name, img_dir):
    for ext in ['jpg', 'jpeg', 'png']:
        img_path = os.path.join(img_dir, f"{img_name}.{ext}")
        if os.path.exists(img_path):
            return img_path
    raise FileNotFoundError(f"Image file for {img_name} not found with extensions: jpg, jpeg, png")

# Function to load and preprocess images
def load_images(df, img_dir, img_size=(128, 128)):
    images = []
    labels = []
    for _, row in df.iterrows():
        try:
            # Find the image with supported extensions
            img_path = find_image_file(os.path.splitext(row['image'])[0], img_dir)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
            img = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalize
            images.append(img)
            labels.append(row['label'])
        except FileNotFoundError as e:
            print(e)  # Print warning if an image is missing
    return tf.convert_to_tensor(images), tf.convert_to_tensor(labels)

# Load training and validation images
IMG_SIZE = (128, 128)
x_train, y_train = load_images(train_data, images_dir, IMG_SIZE)
x_val, y_val = load_images(val_data, images_dir, IMG_SIZE)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=len(label_encoder.classes_))
y_val = to_categorical(y_val, num_classes=len(label_encoder.classes_))

# Step 3: Define the CNN Model
model = Sequential([
    # Convolutional layers
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    # Flatten the feature maps
    Flatten(),
    
    # Fully connected layers
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')  # Output layer
])

# Step 4: Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 5: Data Augmentation for Training
data_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Step 6: Train the Model
batch_size = 32
epochs = 100

history = model.fit(
    data_gen.flow(x_train, y_train, batch_size=batch_size),
    validation_data=(x_val, y_val),
    epochs=epochs,
    verbose=1
)

# Step 7: Save the Model
model.save(r"E:\Data set\vehicle dataset\train\vehicle_classification_model.h5")
print("Model saved!")

# Step 8: Evaluate the Model
val_loss, val_accuracy = model.evaluate(x_val, y_val)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Step 9: Plot Accuracy vs Epoch
plt.figure(figsize=(12, 6))

# Plot training accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy vs Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss vs Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Step 1: Load the Trained Model
model_path = r"E:\Data set\vehicle dataset\train\vehicle_classification_model.h5"
model = load_model(model_path)
print("Model loaded successfully!")

# Step 2: Load Label Encoder to Decode Predictions
csv_path = r"E:\Data set\vehicle dataset\train\annotations.csv"
data = pd.read_csv(csv_path)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])
class_names = label_encoder.classes_
print("Labels loaded successfully!")

# Step 3: Define Image Preprocessing Function
def preprocess_image(image_path, img_size=(128, 128)):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions for batch
    return img, img_array

# Step 4: Define Function to Predict Batch of Images
def predict_images_in_folder(folder_path):
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        img, img_array = preprocess_image(image_path)
        prediction = model.predict(img_array)
        predicted_class = np.argmax(prediction)
        predicted_label = class_names[predicted_class]
        confidence = np.max(prediction) * 100
        
        # Display the image and prediction
        plt.imshow(img)
        plt.title(f"{image_file}\nPrediction: {predicted_label}\nConfidence: {confidence:.2f}%")
        plt.axis("off")
        plt.show()
        
        print(f"Image: {image_file}")
        print(f"Predicted Class: {predicted_label}")
        print(f"Confidence: {confidence:.2f}%")
        print("-" * 50)

# Step 5: Test with Images in a Folder
test_folder_path = r"D:\chandira\model test\Test set\Test set"  # Change this to your folder path
predict_images_in_folder(test_folder_path)


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import requests
import serial  
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import time

# Load the Trained Model
model_path = r"G:\Mechatronics\vehicle dataset\train\vehicle_classification_model.h5"
model = load_model(model_path)

# Load Label Encoder
csv_path = r"G:\Mechatronics\vehicle dataset\train\annotations.csv"
data = pd.read_csv(csv_path)
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])
class_names = label_encoder.classes_

# ESP32-CAM Stream URL
ESP32_CAM_URL = "http://192.168.1.4:81/stream"

# Initialize Serial Communication with Arduino
arduino = serial.Serial(port="COM8", baudrate=9600, timeout=1)

def preprocess_image(frame, img_size=(128, 128)):
    img = cv2.resize(frame, img_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


def detect_from_esp32():
    try:
        stream = requests.get(ESP32_CAM_URL, stream=True, timeout=5)
        if stream.status_code != 200:
            print("Error: Could not connect to ESP32-CAM.")
            return
    
        bytes_stream = b''
        buffer_size_limit = 65536  # Limit buffer size to prevent lag

        for chunk in stream.iter_content(chunk_size=1024):
            bytes_stream += chunk
            if len(bytes_stream) > buffer_size_limit:
                bytes_stream = bytes_stream[-buffer_size_limit:]  # Trim buffer

            a = bytes_stream.find(b'\xff\xd8')  # Start of JPEG
            b = bytes_stream.find(b'\xff\xd9')  # End of JPEG

            if a != -1 and b != -1:
                jpg = bytes_stream[a:b+2]
                bytes_stream = bytes_stream[b+2:]
                
                if len(jpg) == 0:
                    print("Warning: Empty frame received. Skipping.")
                    continue
                
                frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
                
                if frame is None:
                    print("Warning: Failed to decode frame. Skipping.")
                    continue

                img_array = preprocess_image(frame)
                prediction = model.predict(img_array, verbose=0)
                predicted_class = np.argmax(prediction)
                predicted_label = class_names[predicted_class]
                confidence = np.max(prediction) * 100

                # Send detected vehicle type to Arduino
                arduino.write(predicted_label.encode() + b'\n')
                print(f"Detected: {predicted_label}, Confidence: {confidence:.2f}%")

                # Draw bounding box and label
                height, width, _ = frame.shape
                cv2.rectangle(frame, (50, 50), (width - 50, height - 50), (0, 255, 0), 2)
                cv2.putText(frame, f"{predicted_label} ({confidence:.2f}%)", (60, 90),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

                cv2.imshow("ESP32 Vehicle Detection", frame)

                # Press 'q' to exit
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        cv2.destroyAllWindows()

    except requests.exceptions.RequestException as e:
        print(f"Network error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    finally:
        if arduino.is_open:
            arduino.close()

# Start Detection from ESP32-CAM
detect_from_esp32()


